In [11]:
import os

In [12]:
api_key = os.getenv("GEMINI_API_KEY")

In [13]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent, SequentialAgent, ParallelAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner, InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.tools.tool_context import ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.genai import types

print("Sessions and context engineering (3a) and MAS (1b) and Tools (2a) ADK imports")

Sessions and context engineering (3a) and MAS (1b) and Tools (2a) ADK imports


In [14]:
APP_NAME = "Wellness Suite"
USER_ID = "Hector"
SESSION = "default"
# USER_PROFILE = {
#     "name": USER_ID,
#     "age": None,
#     "weight": None,
#     "height": None,
#     "goals": [],
#     "injuries": [],
#     "dietary_restrictions": []
# }

MODEL_NAME = "gemini-2.5-flash-lite"

In [15]:
retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=7, # Delay multiplier
    initial_delay=1, # initial delay before first retry (seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [16]:
# Set up Session Management (stored in RAM)
session_service = InMemorySessionService()

In [17]:
# Helper functions for running a session
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")

In [18]:
# Specialist Agents

# Fitness Trainer Agent: focuses on fitness
fitness_trainer_agent = Agent(
    name="FitnessTrainer",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are an expert fitness trainer with deep knowledge of human physiology.
    Please provide a detailed fitness training plan for the user that is customized for the following:
    1. Their current level of fitness
    2. To help them achieve their fitness goals
    3. Takes into consideration their injury history
    4. Sets realistic time frames for developing lifting technique and expertise
    """,
    tools=[],
    output_key="fitness_plan"
)

# Nutritionist Agent: focuses on nutrition
nutritionist_agent = Agent(
    name="Nutritionist",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are an expert nutritionist with deep knowledge of diet and human metabolism.
    Please provide a detailed nutrition plan for the user that is customized for the following:
    1. Their dietary restrictions
    2. To help them achieve their weight and fitness goals
    3. Suggest meal plans that align with their preferences
    4. Ensure that the nutrition plan meets the macronutrient and micronutrient needs to achieve their goals
    """,
    tools=[],
    output_key="nutrition_plan"
)

In [19]:
# Aggregator Agent

aggregator_agent = Agent(
    name="WellnessAggregator",
    model=Gemini(
        model=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""Combine the fitness plan and nutrition plan into a single wellness plan tailored for the user.
    Focus on the synergy between the fitness and nutrition aspects to maximize the user's health and wellness outcomes.

    **Fitness Plan:**
    {fitness_plan}

    **Nutrition Plan:**
    {nutrition_plan}

    Your final wellness plan should seamlessly integrate both components and provide clear, actionable recommendations.
    """,
    output_key="wellness_plan"
)

In [20]:
# Parallel Agent to run both specialists simultaneously
parallel_wellness_team = ParallelAgent(
    name="ParallelWellnessTeam",
    sub_agents=[fitness_trainer_agent, nutritionist_agent],
)

In [21]:
# Sequential Agent to run the parallel team first, then run the aggregator
wellness_pipeline = SequentialAgent(
    name="WellnessSystem",
    sub_agents=[parallel_wellness_team, aggregator_agent],
)

In [22]:
# Intake Agent

intake_agent = Agent(
    name="WellnessIntakeAgent",
    model=Gemini(
        model_name=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are an expert wellness assistant agent with concise yet kind verbiage.
    Ensure that the user provides necessary personal information for fitness and nutrition advice by asking questions about their health.
    Craft your questions to collect the most essential information (e.g. weight, height, dietary preferences, injury history, general fitness level).
    """,
    tools=[],
    output_key="intake_questions",
)

In [27]:
# Centralized root Wellness Coordinator Agent to provide holistic wellness plan

root_agent = Agent(
    name="WellnessCoordinatorAgent",
    model=Gemini(
        model_name=MODEL_NAME,
        retry_options=retry_config
    ),
    instruction="""You are the Wellness Coordinator Agent.
    Your goal is to assist users in improving their overall wellness by orchestrating the following workflow.
    1. First, ascertain whether or not you have the user's necessary personal information to provide customized fitness and nutrition advice.
    2. If not, call the `intake_agent` tool to craft questions to obtain the necessary personal information.
    3. Once you have sufficient information, call the `wellness_pipeline` tool to create a wellness plan for the user.
    4. Present the results of the wellness plan to the user.
    """,
    tools=[
        AgentTool(intake_agent),
        AgentTool(wellness_pipeline)
    ],
)

In [28]:
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

In [25]:
await run_session(
    runner,
    [
        """Hello! My name is Hector.
        """
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Hello! My name is Hector.
        


gemini-2.5-flash-lite >  Hello Hector! It's great to connect with you. To help me create the most effective and personalized wellness plan for you, I need a little more information.

Could you please tell me:

*   **Your age, height, and current weight?**
*   **What's your general fitness level or how active are you typically?** (e.g., sedentary, moderately active, very active)
*   **Do you have any dietary preferences or restrictions I should know about?** (e.g., vegetarian, gluten-free, allergies)
*   **And finally, are there any past or current injuries that I should be aware of when designing your plan?**

Once I have these details, I can start crafting something truly special for you!


In [26]:
await run_session(
    runner,
    [
        """I am 30 years old, 5 foot 6 inches tall, and 170 pounds.
        I am currently sedentary but have prior powerlifting experience (about 7 years ago).
        I have no particular dietary preferences or restrictions.
        I tore my left rotator cuff and had it surgically repaired.
        """
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > I am 30 years old, 5 foot 6 inches tall, and 170 pounds.
        I am currently sedentary but have prior powerlifting experience (about 7 years ago).
        I have no particular dietary preferences or restrictions.
        I tore my left rotator cuff and had it surgically repaired.
        


gemini-2.5-flash-lite >  Hector, this is fantastic news! I've put together a comprehensive wellness plan specifically tailored to your unique profile, taking into account your age, height, weight, prior powerlifting experience, current activity level, and most importantly, your surgically repaired left rotator cuff.

This plan is all about a gradual and safe return to strength, rebuilding your muscles, improving your cardiovascular health, and ensuring your shoulder stays healthy. We'll be working towards achieving a healthier body composition together!

Here's a snapshot of what your plan entails:

*   **Nutritional Focus:** We're looking at approximately **2250 calories per day**, with a strong emphasis on protein (180-220g) to support muscle repair, good carbohydrates for fuel, and healthy fats for overall well-being. Hydration is key, so aim for at least a gallon of water daily.
*   **Fitness Approach:** Your rotator cuff health is paramount! We'll start with foundational movements

In [29]:
await run_session(
    runner,
    [
        """Can you go into detail about each of the phases of the wellness plan?
        """
    ],
    "stateful-agentic-session",
)


 ### Session: stateful-agentic-session

User > Can you go into detail about each of the phases of the wellness plan?
        


gemini-2.5-flash-lite >  Absolutely, Hector! Let's break down each phase of your personalized wellness plan in detail.

---

### Phase 1: Foundation & Rehabilitation (Weeks 1-4)

**Focus:** The primary goal here is to re-establish basic movement patterns, activate stabilizing muscles, improve both shoulder mobility and strength, and build a foundational cardiovascular base. This phase places a strong emphasis on **technique development and executing movements without pain.**

**Weekly Schedule:**

*   **Monday:** Full Body Strength & Mobility (Focus: Shoulder Health)
*   **Tuesday:** Light Cardio & Core
*   **Wednesday:** Active Recovery / Rest
*   **Thursday:** Full Body Strength & Mobility (Focus: Shoulder Health)
*   **Friday:** Light Cardio & Core
*   **Saturday:** Active Recovery / Rest
*   **Sunday:** Rest

**Detailed Breakdown:**

**1. Strength & Mobility (Monday & Thursday)**

*   **Nutrition Focus:** Maintain your target of approximately **2250 calories** with a strong emphasi